In [27]:
import ipywidgets as widgets
from IPython.display import display
import subprocess

# Function to update the SMILES file with user input
def update_smiles_file(smiles):
    with open('mol2mol.smi', 'w') as file:
        file.write(smiles + '\t' + 'UserInput')

# Function to run REINVENT
#def run_reinvent():
#    cmd = 'python ./REINVENT4/reinvent/Reinvent.py -l mol2mol.log mol2mol.toml'
#    process = subprocess.run(cmd, shell=True, text=True, capture_output=True)
#    print(process.stdout)  # Optionally print any output for debugging
#    return process.stdout

# Function to run REINVENT
def run_reinvent():
    cmd = 'python ./REINVENT4/reinvent/Reinvent.py -l mol2mol.log mol2mol.toml'
    print("Running command:", cmd)  # Print the command to verify it
    process = subprocess.run(cmd, shell=True, text=True, capture_output=True)
    print("STDOUT:", process.stdout)  # Print standard output for debugging
    print("STDERR:", process.stderr)  # Print errors to identify issues
    return process.stdout

# Widget setup
smiles_input = widgets.Text(
    value='',
    placeholder='Type your SMILES here',
    description='SMILES:',
    disabled=False
)
button = widgets.Button(description="Generate Molecules")
output = widgets.Output()

display(smiles_input, button, output)

# Button click event handler
def on_button_clicked(b):
    with output:
        print("Generating molecules for SMILES: ", smiles_input.value)
        # Update the SMILES file with the user's input
        update_smiles_file(smiles_input.value)
        # Call REINVENT to generate molecules
        run_reinvent()

button.on_click(on_button_clicked)


Text(value='', description='SMILES:', placeholder='Type your SMILES here')

Button(description='Generate Molecules', style=ButtonStyle())

Output()

In [1]:
import subprocess
import os

# Print the current working directory for reference
print("Current Directory:", os.getcwd())

# Define the command with the full path to the reinvent executable
cmd = 'python ./REINVENT4/reinvent/Reinvent.py --help'

# Run the command
process = subprocess.run(cmd, shell=True, text=True, capture_output=True)

# Print the output and any errors
print("Output:", process.stdout)
print("Errors:", process.stderr)


Current Directory: C:\Users\chris\OneDrive\Documents\GitHub\Small-Molecule-Optimization
Output: usage: Reinvent.py [-h] [-f FORMAT] [-d DEV] [-l FILE] [--log-level LEVEL]
                   [-s N] [--dotenv-filename FILE]
                   [--enable-rdkit-log-levels LEVEL [LEVEL ...]] [-V]
                   [FILE]

REINVENT: a molecular design tool for de novo design, scaffold hopping,
R-group replacement, linker design, molecule optimization, and others

positional arguments:
  FILE                  Input configuration file with runtime parameters
                        (default: None)

options:
  -h, --help            show this help message and exit
  -f FORMAT, --config-format FORMAT
                        File format of the configuration file: toml, json
                        (default: toml)
  -d DEV, --device DEV  Device to run on: cuda, cpu. Overwrites setting in the
                        configuration file. (default: None)
  -l FILE, --log-filename FILE
                 